In [19]:
import itertools
from concurrent.futures.thread import ThreadPoolExecutor
import requests
import pandas as pd
from sqlalchemy import MetaData, create_engine
import networkx as nx
import math

# System Data

In [5]:
engine = create_engine('sqlite:///eve.db')
connection = engine.connect()
metadata = MetaData()

In [ ]:
def system(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/systems/{id}/?datasource=tranquility&language=en').json()
    systemdict = {}
    systemdict['system_id'] = data['system_id']
    systemdict['name'] = data['name']
    systemdict['constellation_id'] = data['constellation_id']
    systemdict['security_status'] = data['security_status']
    systemdict['x'] = data['position']['x']
    systemdict['y'] = data['position']['y']
    systemdict['z'] = data['position']['z']
    return systemdict

def getsystems(systems):
    with ThreadPoolExecutor() as tp:
        results = tp.map(system,systems)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/systems/?datasource=tranquility')
systems = r.json()
syslist = getsystems(systems)
assert len(syslist) == len(systems)
df = pd.DataFrame(syslist)
df.to_sql('systems', con = connection, index = False, if_exists='replace')

# Constellations

In [49]:
def constellation(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/constellations/{id}/?datasource=tranquility&language=en').json()
    constellationdict = {}
    constellationdict['constellation_id'] = data['constellation_id']
    constellationdict['name'] = data['name']
    constellationdict['region_id'] = data['region_id']
    return constellationdict

def getconstellations(constellations):
    with ThreadPoolExecutor() as tp:
        results = tp.map(constellation,constellations)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/constellations/?datasource=tranquility')
constellations = r.json()
constlist = getconstellations(constellations)
assert len(constlist) == len(constellations)
df = pd.DataFrame(constlist)
df.to_sql('constellations', con = connection, index = False, if_exists='replace')

# Regions

In [38]:
def region(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/regions/{id}/?datasource=tranquility&language=en').json()
    regiondict = {}
    regiondict['region_id'] = data['region_id']
    regiondict['name'] = data['name']
    return regiondict

def getregions(regions):
    with ThreadPoolExecutor() as tp:
        results = tp.map(region,regions)
        return [result for result in results]


r = requests.get('https://esi.evetech.net/latest/universe/regions/?datasource=tranquility')
regions = r.json()
reglist = getregions(regions)
assert len(reglist) == len(regions)
df = pd.DataFrame(reglist)
df.to_sql('regions', con = connection, index = False, if_exists='replace')

# System Kills

In [47]:
data = requests.get('https://esi.evetech.net/latest/universe/system_kills/?datasource=tranquility').json()
df = pd.DataFrame(data)
df = df.reindex(columns = ['system_id','npc_kills','ship_kills','pod_kills'])
df.to_sql('system_kills', con = connection, index= False, if_exists = 'replace')

# Thera Connections

In [69]:
data = requests.get('https://www.eve-scout.com/api/wormholes').json()
theradata = []
for tdict in data:
    theradata.append({
        'system_id':tdict['destinationSolarSystem']['id'],
        'type':tdict['sourceWormholeType']['name']
     })
df = pd.DataFrame(theradata)
df.to_sql('thera', con = connection, index = False, if_exists='replace')

# Stargates

In [82]:
def stargate(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/systems/{id}/?datasource=tranquility&language=en').json()
    try:
        return data['stargates']
    except KeyError:
        return []

def getstargates(stargates):
    with ThreadPoolExecutor() as tp:
        results = tp.map(stargate,stargates)
        rlist = [result for result in results]
        return list(itertools.chain(*rlist))


r = requests.get('https://esi.evetech.net/latest/universe/systems/?datasource=tranquility')
stargates = r.json()
gatelist = getstargates(stargates)
df = pd.DataFrame(gatelist)
df.columns = ['gate_id']
df.to_sql('stargates', con = connection, index = False, if_exists='replace')

,0
0,50000056
1,50000057
2,50000058
3,50000067
4,50000068
...,...
13751,50016363
13752,50016365
13753,50016352
13754,50016370


# Connections


In [14]:
def sconnection(id):
    data = requests.get(f'https://esi.evetech.net/latest/universe/stargates/{id}/?datasource=tranquility').json()
    return {
        'source':data['system_id'],
        'destination':data['destination']['system_id']
    }

def getconnections(stargates):
    with ThreadPoolExecutor() as tp:
        results = tp.map(sconnection,stargates)
        return [result for result in results]


stargates = connection.execute('select gate_id from stargates').fetchall()
stargates = [y for x in stargates for y in x]
conlist = getconnections(stargates)
assert len(conlist) == len(stargates)
df = pd.DataFrame(conlist)
df.to_sql('connections', con = connection, index = False, if_exists='replace')

50000056
5000005750000068

50000067
50000058
50000343
50000342
50001729
50001730
50000345
50000344
50002078
50001739
50001740
50001741
50001742
50002079
50002085
5000208450002086

50002191
50002192
50002193
50002194
50000110
50000111
50000025
5000117650001177
50000550
50000112

50000551
50001442
50001443
50001444
50001445
50001446
50001576
50001577
50001578
50001595
5000159450001596

500003005000029850000297


5001396750002093

50000423
50001197
50000424
50001608
50016504
50016506
50001979
50001978
50002892
50002890
50003795
50003796
50003797
50004483
50004485
50004486
50004894
50004929
50004930
50004931
50004932
50013575
50005226
50000278
50005227
50005545
50005546
50005547
50005548
50000467
50004893
50000664
50000665
50002000
5000305150002001

50004085
50004086
50013573
50004229
50004230
50013572
50005475
50000638
50000639
50000468
50000640
50000641
50000852
50000853
50000854
50001492
50000276
50000277
50003794
50004484
50004375
50003544
50003108
50004376
50013700
50004895
50000860
5

In [21]:
noduplicates = connection.execute('SELECT DISTINCT source_id, destination_id FROM connections t1 WHERE t1.source_id > t1.destination_id OR NOT EXISTS (SELECT * FROM connections t2 WHERE t2.source_id = t1.destination_id AND t2.destination_id = t1.source_id)').fetchall()
df = pd.DataFrame(noduplicates)
df.columns = ['source_id','destination_id']
df.to_sql('connections',con = connection, index=False, if_exists='replace')

In [22]:
eve = nx.Graph()
eve.add_edges_from(noduplicates)

In [23]:
query = """
SELECT s.system_id, s.name as system, security_status, r.name as region, npc_kills, pod_kills, ship_kills, x, y, z
FROM systems AS s
         INNER JOIN constellations c ON c.constellation_id = s.constellation_id
         INNER JOIN regions r ON r.region_id = c.region_id
         INNER JOIN system_kills sk ON s.system_id = sk.system_id
         """
test = connection.execute(query)
df = pd.DataFrame(test, columns=test.keys()).sort_values(by=['npc_kills'], ascending=False)
df

,system_id,system,security_status,region,npc_kills,pod_kills,ship_kills,x,y,z
650,30001408,Ruvas,0.780765,Lonetrek,1639,0,1,-1.753867e+17,9.823873e+16,1.145402e+17
2201,30004142,Hikansog,0.811302,Kador,1472,0,1,-2.352238e+17,5.373238e+16,-1.218548e+17
2204,30004145,Hapala,0.882674,Kador,1280,0,0,-2.337914e+17,6.044064e+16,-1.342971e+17
1789,30003490,Khopa,0.810458,Domain,1121,0,0,-1.934307e+17,6.119444e+16,-6.822568e+16
908,30002072,Inder,0.510338,Metropolis,1062,0,0,-1.341579e+17,5.288728e+16,5.817859e+16
...,...,...,...,...,...,...,...,...,...,...
2426,30004711,PR-8CA,-0.026540,Delve,0,0,1,-3.992338e+17,5.403608e+16,-2.248272e+17
1049,30002333,IRD-HU,-0.434728,Etherium Reach,0,1,0,1.830576e+17,6.841054e+16,2.537526e+16
2424,30004709,KFIE-Z,-0.003493,Delve,0,5,7,-3.896306e+17,3.589244e+16,-2.118520e+17
2741,30005297,Ouelletta,0.418579,Verge Vendor,0,1,9,-2.396225e+17,3.760533e+16,5.922469e+16


In [24]:
query = """
SELECT thera.system_id, name, x, y, z from thera
INNER JOIN systems s ON s.system_id = thera.system_id
"""
theradata = connection.execute(query)
thera = pd.DataFrame(theradata, columns= theradata.keys())
thera

,system_id,name,x,y,z
0,30000030,Kasrasi,-5.993228e+16,4.323341e+16,-4.156472e+16
1,30000099,Arena,-6.854863e+16,7.063457e+16,-7.632508e+16
2,30000656,ARG-3R,3.280840e+16,5.144243e+15,-1.395352e+17
3,30001305,QHJ-FW,-1.353826e+17,1.047674e+17,3.426248e+17
4,30001683,Nasesharafa,-1.885264e+17,9.938116e+16,-1.201786e+17
5,30002309,N06Z-Q,1.797160e+17,8.728983e+16,3.451965e+16
6,30002404,Half,-5.283596e+16,-9.294310e+14,-4.171568e+15
7,30003043,Uphene,-1.999264e+17,1.323508e+16,2.024206e+16
8,30003915,Aurejet,-3.103081e+17,2.596601e+16,-1.321063e+17
9,30004020,ED-L9T,-3.026546e+17,7.054827e+16,-2.389375e+17


In [25]:
closest_gate = []
for row in df.itertuples():
    shortest = int(10000000)
    closest = int(0)
    for trow in thera.itertuples():
        try:
            pathlength = len(nx.shortest_path(eve,source = row.system_id, target = trow.system_id))
            if shortest > pathlength:
                shortest = pathlength
                closest = trow.name
        except:
            continue
    closest_gate.append(
        {
            'closest_gate': closest,
            'gate_distance': shortest
        }
    )

In [26]:
len(closest_gate)
df['closest_gate'] = [x['closest_gate'] for x in closest_gate]
df['gate_distance'] = [x['gate_distance'] for x in closest_gate]
df

,system_id,system,security_status,region,npc_kills,pod_kills,ship_kills,x,y,z,closest_gate,gate_distance
650,30001408,Ruvas,0.780765,Lonetrek,1639,0,1,-1.753867e+17,9.823873e+16,1.145402e+17,Uphene,18
2201,30004142,Hikansog,0.811302,Kador,1472,0,1,-2.352238e+17,5.373238e+16,-1.218548e+17,Nasesharafa,14
2204,30004145,Hapala,0.882674,Kador,1280,0,0,-2.337914e+17,6.044064e+16,-1.342971e+17,Nasesharafa,17
1789,30003490,Khopa,0.810458,Domain,1121,0,0,-1.934307e+17,6.119444e+16,-6.822568e+16,Nasesharafa,10
908,30002072,Inder,0.510338,Metropolis,1062,0,0,-1.341579e+17,5.288728e+16,5.817859e+16,Half,11
...,...,...,...,...,...,...,...,...,...,...,...,...
2426,30004711,PR-8CA,-0.026540,Delve,0,0,1,-3.992338e+17,5.403608e+16,-2.248272e+17,ED-L9T,19
1049,30002333,IRD-HU,-0.434728,Etherium Reach,0,1,0,1.830576e+17,6.841054e+16,2.537526e+16,N06Z-Q,7
2424,30004709,KFIE-Z,-0.003493,Delve,0,5,7,-3.896306e+17,3.589244e+16,-2.118520e+17,ED-L9T,18
2741,30005297,Ouelletta,0.418579,Verge Vendor,0,1,9,-2.396225e+17,3.760533e+16,5.922469e+16,Uphene,14


In [27]:
def distance(curloc, otherloc):
    dist = math.sqrt(
        (curloc[0] - otherloc[0]) ** 2 + (curloc[1] - otherloc[1]) ** 2 + (curloc[2] - otherloc[2]) ** 2)
    return dist / 149597870691 / 63239.6717


def closestly(indf,thera):
    dists = []
    names = []
    for tup in indf.itertuples():
        tempdist = []
        tempname = []
        for tup2 in thera.itertuples():
            tempdist.append(distance((tup.x, tup.y, tup.z), (tup2.x, tup2.y, tup2.z)))
            tempname.append(tup2.name)
        smallest = tempdist.index(min(tempdist))
        dists.append(tempdist[smallest])
        names.append(tempname[smallest])

    return names, dists

In [31]:
df['closest_ly'], df['distance_ly'] = closestly(df,thera)
df[['system','region','npc_kills','pod_kills','ship_kills','closest_gate','gate_distance','closest_ly','distance_ly']]

,system,region,npc_kills,pod_kills,ship_kills,closest_gate,gate_distance,closest_ly,distance_ly
650,Ruvas,Lonetrek,1639,0,1,Uphene,18,Uphene,13.667925
2201,Hikansog,Kador,1472,0,1,Nasesharafa,14,Nasesharafa,6.904941
2204,Hapala,Kador,1280,0,0,Nasesharafa,17,Nasesharafa,6.485532
1789,Khopa,Domain,1121,0,0,Nasesharafa,10,Nasesharafa,6.835102
908,Inder,Metropolis,1062,0,0,Half,11,Uphene,9.054066
...,...,...,...,...,...,...,...,...,...
2426,PR-8CA,Delve,0,0,1,ED-L9T,19,Mafra,9.976505
1049,IRD-HU,Etherium Reach,0,1,0,N06Z-Q,7,N06Z-Q,2.245310
2424,KFIE-Z,Delve,0,5,7,ED-L9T,18,Mafra,7.931111
2741,Ouelletta,Verge Vendor,0,1,9,Uphene,14,Uphene,6.420354
